In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim


/home/wms29/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [31]:
#Import the data
twitter_training = pd.read_csv('twitter_training.csv')
twitter_validation = pd.read_csv('twitter_validation.csv')
#get only text and label cols
twitter_training_clean = twitter_training[['text','label']]
twitter_validation_clean = twitter_validation[['text','label']]

print(twitter_training.head())
#Remove all non alpha characters row by row
#clear nans
twitter_training_clean = twitter_training_clean.dropna()
for index, row in twitter_training_clean.iterrows():
    try:
        twitter_training_clean.at[index, 'text'] = ''.join(e for e in row['text'] if e.isalpha() or e.isspace())
        twitter_training_clean.at[index, 'text'] = row['text'].lower()
        twitter_training_clean.at[index, 'text'] = row['text'].strip()
        twitter_training_clean.at[index, 'text'] = row['text'].split()
        twitter_training_clean.at[index, 'text'] = ' '.join(row['text'])
    except:
        print("Error at index: ", index)
        print(row['text'])
#remove all rows with label = Irrelevant



twitter_validation_clean = twitter_validation_clean.dropna()
for index, row in twitter_validation_clean.iterrows():
    try:
        twitter_validation_clean.at[index, 'text'] = ''.join(e for e in row['text'] if e.isalpha() or e.isspace())
        twitter_validation_clean.at[index, 'text'] = row['text'].lower()
        twitter_validation_clean.at[index, 'text'] = row['text'].strip()
        twitter_validation_clean.at[index, 'text'] = row['text'].split()
        twitter_validation_clean.at[index, 'text'] = ' '.join(row['text'])
    except:
        print("Error at index: ", index)
        print(row['text'])
        
print(twitter_training_clean.head().to_string())
print(twitter_validation_clean.head().to_string())

     id        topic     label  \
0  2401  Borderlands  Positive   
1  2401  Borderlands  Positive   
2  2401  Borderlands  Positive   
3  2401  Borderlands  Positive   
4  2401  Borderlands  Positive   

                                                text  
0  im getting on borderlands and i will murder yo...  
1  I am coming to the borders and I will kill you...  
2  im getting on borderlands and i will kill you ...  
3  im coming on borderlands and i will murder you...  
4  im getting on borderlands 2 and i will murder ...  
                                                     text     label
0     im getting on borderlands and i will murder you all  Positive
1      i am coming to the borders and i will kill you all  Positive
2       im getting on borderlands and i will kill you all  Positive
3      im coming on borderlands and i will murder you all  Positive
4  im getting on borderlands and i will murder you me all  Positive
                                                         

In [3]:
import random
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True


In [4]:
#Create iterators from the pd
from torchtext.legacy import data

TEXT = data.Field(tokenize = 'spacy', include_lengths = True, tokenizer_language = 'en_core_web_sm')
LABEL = data.LabelField(dtype = torch.float)
print(len(twitter_training_clean))
# try:
#     train_data, test_data = torch.utils.data.random_split(twitter_training_clean, [int(len(twitter_training_clean)*0.9) , int(len(twitter_training_clean)*0.1)])
# except:
#     train_data, test_data = torch.utils.data.random_split(twitter_training_clean, [int(len(twitter_training_clean)*0.9) +1, int(len(twitter_training_clean)*0.1)])
val_data = twitter_validation_clean


73996


In [5]:
fields = [('label', LABEL), ('text', TEXT)]
examples = [data.Example.fromlist([row['label'], row['text']], fields) for _, row in twitter_training_clean.iterrows()]


In [37]:
dataset = data.Dataset(examples, fields)
train_data, test_data = dataset.split(split_ratio=0.9, random_state=random.seed(SEED))
dataset_val = data.Dataset([data.Example.fromlist([row['label'], row['text']], fields) for _, row in twitter_validation_clean.iterrows()], fields)
val_data = dataset_val

In [7]:
print(train_data[0].text)

['unk', 'or', 'rainbowsixde', 'or', 'is', 'this', 'no', 'joke', 'image', 'link', 'imgurcomaoltnd', 'and']


In [8]:
TEXT.build_vocab(train_data, max_size=25000, vectors="glove.6B.100d", unk_init=torch.Tensor.normal_)
LABEL.build_vocab(train_data)


In [74]:
BATCH_SIZE = 64
def example_sort_key(example):
    return len(example.text)

train_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, test_data), 
    batch_size=BATCH_SIZE,
    shuffle=True,
    sort_key=example_sort_key,
    device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
)
val_iterator = data.BucketIterator(
    val_data,
    batch_size=BATCH_SIZE,
    shuffle=True,
    sort_key=example_sort_key,
    device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
)
test_iterator_two = data.BucketIterator(
    test_data,
    batch_size=BATCH_SIZE,
    shuffle=True,
    sort_key=example_sort_key,
    device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
)

In [10]:
# Define a simple LSTM model for sentiment analysis
class LSTMModel(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text):
        #convert text to a tensor

        embedded = self.embedding(text)
        output, (hidden, _) = self.lstm(embedded)
        hidden_1D = hidden[-1, :, :]
        return self.fc(hidden_1D)

In [11]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 4  # Binary classification: positive or negative sentiment

model = LSTMModel(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)
 

In [42]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())
print(model.parameters)

<bound method Module.parameters of LSTMModel(
  (embedding): Embedding(25002, 100)
  (lstm): LSTM(100, 256)
  (fc): Linear(in_features=256, out_features=4, bias=True)
)>


In [16]:
#check what percent it got right
def accuracy(model, iterator):
    correct = 0
    total = 0
    model.eval()
    with torch.no_grad():
        for batch in iterator:
            text_tensor = batch.text  # This will be a tuple
            text_tensor = text_tensor[0]  # Accessing the text tensor for the first field
            target_labels = batch.label.type(torch.long)

            predictions = model(text_tensor).squeeze(1)
            _, predicted = torch.max(predictions.data, 1)
            total += target_labels.size(0)
            correct += (predicted == target_labels).sum().item()
    return (100 * correct / total)


In [13]:
import time
def train(model, iterator, optimizer, criterion):
    model.train()
    for batch in iterator:
        text_tensor = batch.text  # This will be a tuple
        text_tensor = text_tensor[0]  # Accessing the text tensor for the first field

        optimizer.zero_grad()
        predictions = model(text_tensor).squeeze(1)

        target_labels = batch.label.type(torch.long)


        loss = criterion(predictions, target_labels)
        loss.backward()
        optimizer.step()



# Evaluation function
def evaluate(model, iterator, criterion):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in iterator:
            text_tensor = batch.text  # This will be a tuple
            text_tensor = text_tensor[0]  # Accessing the text tensor for the first field
            target_labels = batch.label.type(torch.long)

            predictions = model(text_tensor).squeeze(1)
            loss = criterion(predictions, target_labels)
            total_loss += loss.item()
    return total_loss / len(iterator)

# Train the model for multiple epochs
N_EPOCHS = 15
for epoch in range(N_EPOCHS):
    #Time execution
    timer = time.time()

    
    train(model, train_iterator, optimizer, criterion)
    train_loss = evaluate(model, train_iterator, criterion)
    test_loss = evaluate(model, test_iterator, criterion)
    # test_accuracy = accuracy(model, test_iterator)
    print(f'Epoch: {epoch+1}, Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}, Time: {time.time() - timer:.2f}s')


Epoch: 1, Train Loss: 1.2932, Test Loss: 1.3299, Time: 244.68s
Epoch: 2, Train Loss: 0.8676, Test Loss: 1.0556, Time: 240.20s
Epoch: 3, Train Loss: 0.4846, Test Loss: 0.9081, Time: 282.75s
Epoch: 4, Train Loss: 0.2546, Test Loss: 0.8075, Time: 271.05s
Epoch: 5, Train Loss: 0.1672, Test Loss: 0.7625, Time: 271.14s
Epoch: 6, Train Loss: 0.1404, Test Loss: 0.7070, Time: 261.68s
Epoch: 7, Train Loss: 0.1041, Test Loss: 0.6817, Time: 268.25s
Epoch: 8, Train Loss: 0.0956, Test Loss: 0.6839, Time: 278.10s
Epoch: 9, Train Loss: 0.0837, Test Loss: 0.7036, Time: 258.42s
Epoch: 10, Train Loss: 0.0825, Test Loss: 0.7200, Time: 272.63s
Epoch: 11, Train Loss: 0.0816, Test Loss: 0.7194, Time: 280.42s
Epoch: 12, Train Loss: 0.0763, Test Loss: 0.6866, Time: 277.38s
Epoch: 13, Train Loss: 0.0727, Test Loss: 0.6935, Time: 265.77s
Epoch: 14, Train Loss: 0.0672, Test Loss: 0.6757, Time: 267.90s
Epoch: 15, Train Loss: 0.0640, Test Loss: 0.6726, Time: 271.32s


In [1]:
#Test val_data
print(type(val_data))
print(type(val_iterator))
val_accuracy = accuracy(model, val_iterator)
test_accuracy = accuracy(model, test_iterator)
test_two_accuracy = accuracy(model, test_iterator_two)
train_accuracy = accuracy(model, train_iterator)
print(f'Val Accuracy: {val_accuracy:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')
print(f'Test Two Accuracy: {test_two_accuracy:.4f}')
print(f"Train Accuracy: {train_accuracy:.4f}")
print(len(val_data))



NameError: name 'val_data' is not defined

In [2]:
#Test the model on a signle sentence data point
#function to convert a ssetence to a tensor
def convert_sentence_to_tensor(sentence):
    tokenized = [tok for tok in sentence]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed)
    tensor = tensor.unsqueeze(1)
    return tensor
temp_sentence  = 'I want to quit my job and I hate the government. I really hate this '
temp_tensor = convert_sentence_to_tensor(temp_sentence)
pred = model(temp_tensor).unsqueeze(1)

print(pred)
print(torch.argmax(pred))
#print the label
print(LABEL.vocab.itos[torch.argmax(pred)])
# print(LABEL.vocab.stoi['negative'])

NameError: name 'TEXT' is not defined